In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)

In [ ]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   
df = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
def percentile(s):
    return np.sum(s) / len(s)

In [ ]:
stu_groupby = df.groupby('userID').agg({
    'assessmentItemID': 'count',
    'answerCode': percentile
})
stu_groupby.describe()

## (a)

In [ ]:
itemnum_ans = stu_groupby.groupby('assessmentItemID').mean()
itemnum_ans['num_items'] = itemnum_ans.index

In [ ]:
fig, ax = plt.subplots()
sns.regplot(data=itemnum_ans, x='num_items', y='answerCode',
           line_kws={"color": "orange"}, scatter_kws={'alpha':0.6}, ax=ax)

ax.set_title('# of Questions - Answer Rate')
ax.set_xlabel('# of Questions')
ax.set_ylabel('Answer Rate')

## (b)

In [ ]:
itemnum_ans = stu_groupby.groupby('assessmentItemID').mean()
bins = 300
itemnum_ans['bins'] = pd.cut(
    itemnum_ans.index,
    [i * (itemnum_ans.index.max() - itemnum_ans.index.min()) // bins for i in range(bins)]
)

In [ ]:
itemnum_ans = itemnum_ans.groupby('bins').mean()
itemnum_ans['mid'] = list(map(lambda x: (x.left + x.right)//2, itemnum_ans.index))

In [ ]:
fig, ax = plt.subplots()
sns.regplot(data=itemnum_ans, x='mid', y='answerCode',
           line_kws={"color": "orange"}, scatter_kws={'alpha': 0.6}, ax=ax)

ax.set_title(f'# of Items - Answer Rate | bins={bins}')
ax.set_xlabel('# of Items')
ax.set_ylabel('Answer Rate')

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.



In [ ]:
(df == df.sort_values(['userID', 'Timestamp'], ascending = [True, True]))

# FE

In [ ]:
def acc(df_user):
    df_user['acc_rate'] = np.zeros((df_user.shape[0],1))
    a = 0
    b = 0
    for i in sorted(df_user.index):
        b += 1
        a += df_user['answerCode'][i]
        df_user['acc_rate'][i] = a/b
    return df_user['acc_rate']

In [ ]:
from tqdm import tqdm
li = list()
for i in tqdm(np.unique(df['userID'])):
    a = df[df['userID'] == i]
    li += acc(a).to_list()
np.save('acc_list.npy', np.array(li))
    

In [ ]:
df['acc_rate'] = li

In [ ]:
def cut(a,b,s):
    return s[a:b]

df['sheet_id'] = [cut(1,7,i) for i in df['assessmentItemID']]
df['sheet_pr_number'] = [cut(7,11,i) for i in df['assessmentItemID']]

In [ ]:
df

# baseline 정리

In [ ]:
import numpy as np
assess = np.load('input/code/dkt/asset/assessmentItemID_classes.npy')
assess

In [ ]:
knew = np.load('input/code/dkt/asset/KnowledgeTag_classes.npy')
knew

In [ ]:
test = np.load('input/code/dkt/asset/testId_classes.npy')
test

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(knew)

In [ ]:
import pandas as pd
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   
df = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df = df.sort_values(by=["userID", "Timestamp"], axis=0)
columns = ["userID", "assessmentItemID", "testId", "answerCode", "KnowledgeTag"]
group = (
    df[columns]
    .groupby("userID")
    .apply(
        lambda r: (
            r["testId"].values,
            r["assessmentItemID"].values,
            r["KnowledgeTag"].values,
            r["answerCode"].values,
        )
    )
)

In [ ]:
val = group.values

In [ ]:
a, b, c, d  = val[1][0], val[1][1], val[1][2], val[1][3]

In [ ]:
cate_cal = [a, b, c, d]

In [ ]:
len(cate_cal) 

# EDA _ 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sub = pd.read_csv('input/data/sample_submission.csv')
test = pd.read_csv('input/data/test_data.csv')
train = pd.read_csv('input/data/train_data.csv')

In [ ]:
display(train.describe(), test.describe(), sub.describe())

In [ ]:
train.shape, test.shape, sub.shape

In [ ]:
# cold start
cold_start = (list(set(np.unique(test['userID'])) - set(np.unique(train['userID']))))
len(cold_start)

In [ ]:
# cold start sub 에서 비율
len(list(set(np.unique(sub['id'])) & set(cold_start)))

In [ ]:
train.head()

In [ ]:
train_sample = train[train['userID'] == 0]
train_sample.sort_values(by = 'Timestamp')
a = train_sample.groupby(by = ['testId'])['answerCode'].sum()
import seaborn as sns
sns.lineplot(x = range(a.shape[0]), y = a.values)


# FE for columns

In [ ]:
# 오류(SettingWithCopyError 발생)
pd.set_option('mode.chained_assignment', 'raise') # SettingWithCopyError

# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

In [3]:
is_Train = True
import pandas as pd
import numpy as np
from tqdm import tqdm
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}
if is_Train:
    df = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
else:
    df = pd.read_csv('input/data/test_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
df.head()

In [53]:
df['endtime'] = np.roll(df['Timestamp'], -1)
df['totaltime'] = (df['endtime'] - df['Timestamp'])
time_ = list()
for i in df['totaltime']:
    a = i.total_seconds()
    if a > 3600:
        a = 3600
    elif a < 0:
        a = -1
    time_.append(a)
time_[-1] = -1

def grouping(a):
    if a < 0:
        return -1
    if a == 0:
        return 0
    elif 0 < a <= 10:
        return 1
    elif 10 < a <= 20:
        return 2
    elif 20 < a <= 40:
        return 3
    elif 40 < a <= 60:
        return 4
    elif 60 < a <= 120:
        return 5
    elif 120 < a <= 600:
        return a//60 + 3
    else: 
        return a//600 + 13
# (-1~20) feat
df['totaltime'] = [grouping(i) + 1 for i in time_]
if is_Train:
    np.save('/opt/ml/train_col/totaltime.npy', df['totaltime'])
else:
    np.save('/opt/ml/test_col/totaltime.npy', df['totaltime'])


In [ ]:
if is_Train:
    for i in [2,4,5,6,8,9]:
        np.save('/opt/ml/train_col/assessmentItemID_' + str(i) + '.npy', np.array([j[i] for j in df['assessmentItemID']]))
else:
    for i in [2,4,5,6,8,9]:
        np.save('/opt/ml/test_col/assessmentItemID_' + str(i) + '.npy', np.array([j[i] for j in df['assessmentItemID']]))

In [ ]:
from tqdm import tqdm
li = list()
for i in tqdm(np.unique(df['userID'])):
    a = df[df['userID'] == i]
    li += acc(a).to_list()
if is_Train:
    np.save('/opt/ml/train_col/acc_list.npy', np.array(li))
else:
    np.save('/opt/ml/test_col/acc_list.npy', np.array(li))
    

In [5]:
df_train = pd.read_csv('input/data/train_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df_test = pd.read_csv('input/data/test_data.csv', dtype=dtype, parse_dates=['Timestamp'])
df_train = df_train.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df_test = df_test.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [ ]:
set(df_train['assessmentItemID'].tolist()) == set(df_test['assessmentItemID'].tolist())

In [6]:
assess_corr = {}
from tqdm import tqdm
for i in tqdm(set(df_train['assessmentItemID'].tolist())):
    tmp = df_train[df_train.assessmentItemID == i]
    if sum(tmp.answerCode)/tmp.shape[0]>0.5:
        assess_corr[i] = 1
    else:
        assess_corr[i] = 0
    
if is_Train:
    np.save('/opt/ml/train_col/item_correct.npy',[assess_corr[i] for i in df['assessmentItemID']])
else:
    np.save('/opt/ml/test_col/item_correct.npy',[assess_corr[i] for i in df['assessmentItemID']])


100%|██████████| 9454/9454 [24:20<00:00,  6.47it/s]


In [17]:
item_corr = {}
for i in tqdm(set(df_train['testId'].tolist())):
    tmp = df_train[df_train.testId == i]
    if sum(tmp.answerCode)/tmp.shape[0]>0.5:
        item_corr[i] = 1
    else:
        item_corr[i] = 0

if is_Train:
    np.save('/opt/ml/train_col/test_correct.npy',[item_corr[i] for i in df['testId']])
else:
    np.save('/opt/ml/test_col/test_correct.npy',[item_corr[i] for i in df['testId']])


100%|██████████| 1537/1537 [03:59<00:00,  6.40it/s]


In [16]:

list(assess_corr.keys()).index('A060000001')

10067

In [ ]:
# user_corr = {}
# for i in tqdm(set(df_train['userID'].tolist())):
#     tmp = df_train[df_train.userID == i]
#     user_corr[i] = sum(tmp.answerCode)/tmp.shape[0]

# if is_Train:
#     np.save('/opt/ml/train_col/test_correct.npy',[user_corr[i] for i in df['userID']])
# else:
#     np.save('/opt/ml/test_col/test_correct.npy',[user_corr[i] for i in df['userID']])

In [54]:
np.unique(np.load('train_col/totaltime.npy')).astype(int)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])